In [7]:
from pathlib import Path
import json


# Paths
data_dir = Path().cwd() / '../data'

# Load ground truth
with open(data_dir / 'binance_ground_truth.json', 'r') as f:
    binance_ground_truth = json.load(f)
    
with open(data_dir / 'bitpanda_ground_truth.json', 'r') as f:
    bitpanda_ground_truth = json.load(f)

In [ ]:
import requests


# Blockfrost settings
project_id = 'mainnet1nQVUgUPnvLbJPYGJvcUAmKB0T4uurmg'
base_url = 'https://cardano-mainnet.blockfrost.io/api/v0/'
headers = {
    'project_id': project_id
}
# Exchange addresses extracted from withdrawal transactions
exchange_addresses = set()


# Abstract function to handle Blockfrost API response and save data
def handle_blockfrost_api_response(endpoint, parameters={}, paging=False):
    page = 1
    while True:
        parameters.update({'page': page})
        response = requests.get(base_url + endpoint, headers=headers, params=parameters)
        if response.status_code == 200:
            data = response.json()
            if not data:  # No more data to fetch
                break
            # Create necessary directories if they don't exist
            path = Path(data_dir / 'blockfrost' / endpoint)
            path.mkdir(parents=True, exist_ok=True)
            # Save data to a JSON file
            with open(path / f'{page}.json', 'w') as f:
                json.dump(data, f, indent=4)
            page += 1
        else:
            print('Error: ', response.status_code)
            break
        if not paging:
            break


# Fetch and save transactions data for each address
for address in bitpanda_ground_truth['deposit_addrs']:
    handle_blockfrost_api_response(f'addresses/{address}/transactions')

# Fetch and save transaction data for each withdraw transaction
for tx in bitpanda_ground_truth['withdraw_txs']:
    handle_blockfrost_api_response(f'txs/{tx}')

In [ ]:
import networkx as nx


# Create a new directed graph
G = nx.DiGraph()

# Load transactions data from directory and create graph
for filename in os.listdir('data/'):
    with open('data/' + filename, 'r') as f:
        data = json.load(f)
        # Add nodes and edges to the graph
        for transaction in data:
            G.add_edge(transaction['sender'], transaction['receiver'], weight=transaction['amount'])

# Analyze the graph
print('Number of nodes: ', G.number_of_nodes())
print('Number of edges: ', G.number_of_edges())

# Draw the graph
pos = nx.spring_layout(G)

# Determine node colors
node_colors = []
for node in G.nodes():
    if node in bitpanda_ground_truth['deposit_addrs'] or node in exchange_addresses:
        node_colors.append('blue')  # Exchange addresses
    else:
        node_colors.append('red')  # User addresses

nx.draw_networkx_nodes(G, pos, node_color=node_colors)
nx.draw_networkx_labels(G, pos)
nx.draw_networkx_edges(G, pos)
plt.show()